# **Data Preprocessing**

Setelah data dari berbagai sumber berhasil diintegrasikan, langkah selanjutnya adalah prapemrosesan. Tujuan utama dari tahap ini adalah untuk membersihkan data dari anomali atau *outlier* yang dapat mengganggu performa model *machine learning*. *Outlier* adalah titik data yang secara signifikan berbeda dari observasi lainnya.

Proses ini akan dilakukan di dalam **Power BI** dengan memanfaatkan integrasi skrip Python, menggunakan pustaka **PyCaret** yang sangat efisien untuk tugas-tugas AutoML, termasuk deteksi anomali. Dua metode akan digunakan untuk perbandingan: **K-Nearest Neighbors (KNN)** dan **Angle-Based Outlier Detection (ABOD)**.

## **1. Konfigurasi Lingkungan di Power BI**

Sebelum menjalankan skrip PyCaret, Power BI harus dikonfigurasi untuk menggunakan lingkungan Python yang benar di mana PyCaret dan dependensinya telah terpasang.

  * **Implementasi**:

    1.  Buka Power BI Desktop.
    2.  Navigasi ke `File > Options and settings > Options`.
    3.  Di jendela *Options*, pilih `Python scripting`.
    4.  Pada bagian *Detected Python home directories*, pilih direktori instalasi Python yang berisi pustaka `pycaret` dan `pandas`.
    5.  Klik **OK** untuk menyimpan konfigurasi.

## **2. Implementasi di Power Query**

Deteksi dan penghapusan *outlier* dilakukan di dalam Power Query Editor. Ini memungkinkan proses pembersihan data dilakukan sebelum data dimuat ke dalam model data Power BI.

### **A. Memuat Data**

Data iris yang telah digabungkan dari MySQL dan PostgreSQL terlebih dahulu ke dalam Power Query sebagai sebuah tabel.

### **B. Menjalankan Skrip Python**

Dengan tabel data terpilih, navigasi ke `Transform > Run Python script`. Di sinilah logika deteksi anomali menggunakan PyCaret akan dieksekusi. Power Query secara otomatis akan meneruskan data dalam tabel sebagai sebuah DataFrame pandas bernama `dataset`.

## **3. Metode 1: K-Nearest Neighbors (KNN) untuk Deteksi Outlier**

KNN bekerja dengan mengasumsikan bahwa *outlier* adalah titik data yang memiliki jarak yang jauh ke tetangga terdekatnya. PyCaret menyederhanakan implementasi ini menjadi beberapa baris kode.

  * **Implementasi (Kode Python di Power Query)**:

In [ ]:
from pycaret.anomaly import *

# Inisialisasi lingkungan, abaikan kolom non-fitur
anomaly = setup(data=dataset, ignore_features=['id','species'], silent=True)

# Latih model KNN
knn = create_model('knn')

predictions = assign_model(knn)
outliers = predictions[predictions['Anomaly'] == 0]

# Hapus outliers
outliers_removed = predictions[predictions['Anomaly'] == 0]

## **4. Metode 2: Angle-Based Outlier Detection (ABOD)**

ABOD efektif untuk data multidimensi. Metode ini mengidentifikasi *outlier* dengan melihat varians sudut antara vektor perbedaan sebuah titik dengan titik-titik lainnya.

  * **Implementasi (Kode Python di Power Query)**:

In [ ]:
from pycaret.anomaly import *

# Inisialisasi lingkungan
anomaly = setup(data=dataset, ignore_features=['id','species'], silent=True)

# Latih model ABOD dengan asumsi 1% data adalah outlier
abod = create_model('abod', fraction=0.01)

predictions = assign_model(abod) 

#Hapus outliers
outliers_removed = predictions[predictions['Anomaly'] == 0]

## **5. Hasil**

Setelah menjalankan skrip Python untuk masing-masing metode, Power Query akan menampilkan tabel hasil (`outliers_removed`) yang sudah bersih dari *outlier*. Tabel ini kemudian dapat dimuat ke dalam model data Power BI dengan mengklik `Close & Apply`.


In [1]:
# 1. Impor pustaka pandas
import pandas as pd

# 2. Tentukan nama file CSV Anda
file_knn = 'knn.csv'

# 3. Baca file CSV ke dalam DataFrame baru
df_knn = pd.read_csv(file_knn)

print("Data Bersih Hasil KNN:")
print(df_knn.to_string())


Data Bersih Hasil KNN:
      id  petal_length  petal_width  sepal_length  sepal_width          species  Anomaly  Anomaly_Score
0      1           1.4          0.2           5.1          3.5      Iris-setosa        0       0.141421
1      2           1.4          0.2           4.9          3.0      Iris-setosa        0       0.173205
2      3           1.3          0.2           4.7          3.2      Iris-setosa        0       0.264575
3      4           1.5          0.2           4.6          3.1      Iris-setosa        0       0.264575
4      5           1.4          0.2           5.0          3.6      Iris-setosa        0       0.244949
5      6           1.7          0.4           5.4          3.9      Iris-setosa        0       0.387299
6      7           1.4          0.3           4.6          3.4      Iris-setosa        0       0.316228
7      8           1.5          0.2           5.0          3.4      Iris-setosa        0       0.223607
8      9           1.4          0.2      

In [2]:
# 1. Impor pustaka pandas
import pandas as pd

# 2. Tentukan nama file CSV Anda
file_abod = 'abod.csv'

# 3. Baca file CSV ke dalam DataFrame baru
df_knn = pd.read_csv(file_abod)

print("Data Bersih Hasil ABOD:")
print(df_knn.to_string())


Data Bersih Hasil ABOD:
      id  petal_length  petal_width  sepal_length  sepal_width          species  Anomaly          Anomaly_Score
0      1           1.4          0.2           5.1          3.5      Iris-setosa        0    '-556.2514212711023
1      2           1.4          0.2           4.9          3.0      Iris-setosa        0    '-400.0009279294773
2      3           1.3          0.2           4.7          3.2      Iris-setosa        0    '-93.42199195851083
3      4           1.5          0.2           4.6          3.1      Iris-setosa        0    '-99.22922079684693
4      5           1.4          0.2           5.0          3.6      Iris-setosa        0    '-82.17620090369948
5      6           1.7          0.4           5.4          3.9      Iris-setosa        0   '-15.175275265123414
6      7           1.4          0.3           4.6          3.4      Iris-setosa        0   '-29.098910332137837
7      8           1.5          0.2           5.0          3.4      Iris-setosa 